In [11]:
import pandas as pd
import numpy as np
from num2words import num2words
import re
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

## Function 1: pluralize_gift()

In [12]:
#helpful functions
obj_1 = "foot"
obj_2 = "baby"
obj_3 = "tree"

obj_1.find("oo")
obj_2[-1]
obj_3.find("oo")

obj_1.replace("oo", "ee")
obj_2.replace("y", "ies")
obj_3 + "s"

#pluralize gift function
def pluralize_gift(gift):
  """
  Returns plural of a noun
  
  Parameters
  ----------
  gift: str
    A noun
    
  Return
  ------
  str
    Plural version
  """

  if gift.find("oo") != -1:
    gift = gift.replace("oo", "ee")
  elif gift[-1] == "y":
    gift = gift.replace("y", "ies")
  else:
    gift = gift + "s"

  return gift

In [13]:
# Should work
pluralize_gift("goose")

# Will work if your function is vectorized! 
vec_pluralize_gift = np.vectorize(pluralize_gift)
vec_pluralize_gift(xmas['Gift.Item'])

array(['partridges', 'doves', 'hens', 'birds', 'rings', 'geese', 'swans',
       'maids', 'ladies', 'lords', 'pipers', 'drummers'], dtype='<U10')

## Function 2: make_phrase()

In [14]:
def make_phrase(num, num_word, item, verb, adjective, location):
  """
  Returns a phrase
  
  Parameters
  ----------
  num: int
    The day
  num_word: str
    Number of items expressed as a word
  item: str
    A noun representing the item
  verb: str
    A verb describing an action
  adjective: str
    An adjective to describe the item
  location: str
    A noun indicating the location
  
  Return
  ------
  str
    A concatenated phrase in the format "num_word adjective item verb location"
  """
  
  ## Step 1: Replace NAs with blank strings
  verb = pd.Series([verb]).fillna("").iloc[0]
  adjective = pd.Series([adjective]).fillna("").iloc[0]
  location = pd.Series([location]).fillna("").iloc[0]
  
  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  ### Hint: call the function you created above!
  if num > 1:
    item = pluralize_gift(item)
  else:
    item
  
  ## Step 3: Figure out if a gift item starts with a vowel
  vowels = "aeiou"
  starts_with_vowel = item[0] in vowels
  
  ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel
  # replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
  if num == 1:
        if starts_with_vowel:
            num_word = "an"
        else: num_word = "a"
  
  ## Step 5: Put all of the pieces together into one string and return!
  phrase = " ".join([num_word, adjective, item, verb, location]).strip()
  #eliminate extra white space
  phrase = re.sub(r'\s+', ' ', phrase)
  return phrase

In [15]:
# day word column
num_to_word = {
    1: "one",
    2: "two",
    3: "three",
    4: "four",
    5: "five",
    6: "six",
    7: "seven",
    8: "eight",
    9: "nine",
    10: "ten",
    11: "eleven",
    12: "twelve"
  }
xmas["Num.Word"] = xmas["Day"].map(num_to_word)

## Test Your Function

Make sure to try your function out on small examples and on the xmas data.

Then, use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day.

In [16]:
#test 1
print(make_phrase(10, "ten", "lord", "a-leaping", "", ""))
#new column
xmas["Full.Phrase"] = xmas.apply(lambda row: make_phrase(
    row["Day"],
    row["Num.Word"],
    row["Gift.Item"],
    row["Verb"],
    row["Adjective"],
    row["Location"]), axis = 1
    )

ten lords a-leaping
ten lords a-leaping


Function 3: sing_day()
Write a function called sing_day() that takes as input:

A dataset (input as a dataframe)

A number indicating which day to sing about (input as an integer)

The name of a column in the dataset that contains the phrases for each day (input as an tidy name)

In [17]:
def sing_day(dataset, num, phrase_col):
  """
  Returns part of the song
  
  Parameters
  ----------
  dataset: str
    A dataset
  num: int
    number that indicates what day to sing about
  phrase_col: str
    The name of a column in the dataset that contains the phrases for each day
  Return
  ------
  str
    Sings the song for a specific day
  """
  
  # Step 1: Setup the intro line
  num_word = num2words(num, to="ordinal")  # convert "1" to "first" etc.
  intro = "On the " + num_word + " day of Christmas, my true love sent to me:"
  
  # Step 2: Sing the gift phrases
  # Hint: What order are they gifts sung in each day?
  phrases = dataset.loc[dataset['Day'] <= num, phrase_col].iloc[::-1].tolist()

  #white space
  phrases = [phrase.strip() for phrase in phrases]

  #join the phrases and add commas and returns
  if len(phrases) > 1:
        song_line = ',\n'.join(phrases[:-1]) + ', and\n' + phrases[-1]
  elif len(phrases) == 1:
        song_line = phrases[0]
  else:
        song_line = ""

  #combine
  full_song = f"{intro}\n{song_line}."

  return full_song

Test your function
Use this code to show the function works:

In [18]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves, and
a partridge in a pear tree.
On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves, and
a partridge in a pear tree.


## Use Your Functions!
Run appropriate code to output the lyrics for the entire 12 Days of Christmas song.

In [19]:
#print the song & extras
def sing_line(data):
    for day in range (1,13):
        print(sing_day(data, day, "Full.Phrase"))
        # print an empty line
        print("\n")

print(sing_line(xmas))

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.


On the second day of Christmas, my true love sent to me:
two turtle doves, and
a partridge in a pear tree.


On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, and
a partridge in a pear tree.


On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves, 

In [20]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
#map day word column
xmas2["Day.Word"] = xmas2["Day"].map(num_to_word)
#map full phrase column
xmas2["Full.Phrase"] = xmas2.apply(lambda row: make_phrase(
    row["Day"],
    row["Day.Word"],
    row["Gift.Item"],
    row["Verb"],
    row["Adjective"],
    row["Location"]), axis = 1
    )
#print surprise song!
print(sing_line(xmas2))

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.


On the second day of Christmas, my true love sent to me:
two meal points, and
an email from Cal Poly.


On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points, and
an email from Cal Poly.


On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.


On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.


On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.


On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points, and
an email from Cal Poly.

References & Appendix

- https://numpy.org/doc/stable/reference/generated/numpy.vectorize.html

- https://stackoverflow.com/questions/48072749/remove-spaces-from-words-and-generate-exact-words

- https://www.geeksforgeeks.org/python-map-function/

- https://stackoverflow.com/questions/4071396/how-to-split-by-comma-and-strip-white-spaces-in-python

- https://stackoverflow.com/questions/58657924/unable-to-convert-number-to-words-from-pandas-series-using-num2words-python-libr

AI Statement:

In this lab, I used AI assistance for various aspects of the implementation. Specifically, I utilized AI for:

- Vectorization recommendations for functions such as the use of vectorize() in the numpy library to efficiently vectorize a function.

- Suggestions for the num2words library to convert numbers into ordinal words for the song format.

- General coding recommendations, including string formatting using f-strings (e.g., f"{intro}\n{song_line}."), which helped simplify the process of combining text and variables for output.

- Additional guidance on structuring conditional logic, improving code readability, and debugging potential errors.

Generative AI provided useful suggestions to streamline the coding process and ensure the final implementation was efficient and well-organized.